## Productos
- `codigo`: 4 digitos
- `nombre`: 1 a 100 caracteres
- `precio`: 10 a 10000
- `tipo`: 0 a 20 caracteres
- `cantidad`: 0 a 100 
- `valorTotal`: cantidad * precio

In [1]:
class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad):
        if codigo.isdigit() and len(codigo) == 4:
            self.codigo = codigo
        else:
            raise ValueError("Código debe tener 4 dígitos")

        if isinstance(nombre, str) and 1 <= len(nombre) <= 100:
            self.nombre = nombre
        else:
            raise ValueError("Nombre debe tener entre 1 y 100 caracteres")

        self.precio = precio
        self.cantidad = cantidad

        if isinstance(tipo, str) and 0 <= len(tipo) <= 20:
            self.tipo = tipo
        else:
            raise ValueError("Tipo debe tener entre 0 y 20 caracteres")

    @property
    def precio(self):
        return self.__precio

    @precio.setter
    def precio(self, valor):
        if isinstance(valor, (int, float)) and 10 <= valor <= 10000:
            self.__precio = valor
        else:
            print(f"Valor inválido para el precio: {valor}. Se mantiene el precio anterior: {self.__precio}")

    @property
    def cantidad(self):
        return self.__cantidad

    @cantidad.setter
    def cantidad(self, valor):
        if isinstance(valor, int) and 0 <= valor <= 1000:
            self.__cantidad = valor
        else:
            print(f"Valor inválido para la cantidad: {valor}. Se mantiene la cantidad anterior: {self.__cantidad}")

    def valorTotal(self):
        return self.precio * self.cantidad

In [2]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba de la clase Producto

p1 = Producto('0001', 'Coca Cola', 1500, 'gaseosa', 10)

assert p1.codigo == '0001'
assert p1.nombre == 'Coca Cola'
assert p1.precio == 1500

# Calcula el valor total 
assert p1.valorTotal() == 15000 

# Asegura que los valores de precio y cantidad sean validos
p1.precio = -200
assert p1.precio == 1500    # Rechaza el valor negativo

p1.precio = 2000 
assert p1.precio == 2000

p1.cantidad = -5
assert p1.cantidad == 10    # Rechaza el valor negativo

p1.cantidad = 50
assert p1.cantidad == 50
assert p1.valorTotal() == 100000

print("Prueba pasada exitosamente!")

Valor inválido para el precio: -200. Se mantiene el precio anterior: 1500
Valor inválido para la cantidad: -5. Se mantiene la cantidad anterior: 10
Prueba pasada exitosamente!


# Ofertas
Debe permitir aplicar oferctas a codigos espeficicos de productos y a tipos de productos
- `descripcion`: 1 a 100 caracteres
- `codigos`: lista de codigos de productos
- `tipos`: lista de tipo de producto
- `esAplicable(producto, cantidad)`: retorna si la oferta es aplicable a ese producto
- `aplicar(producto, cantidad)`: retorna el precio final del producto con la oferta aplicada


In [3]:
class Oferta:
    def __init__(self, descripcion="Por defecto", codigos=[], tipos=[]):
        if isinstance(descripcion, str) and 1 <= len(descripcion) <= 100:
            self.descripcion = descripcion
        else:
            raise ValueError("Descripcion debe tener entre 1 a 100 caracteres")
        
        if isinstance(codigos, list):
            self.codigos = codigos
        else:
            raise ValueError("Codigos debe ser una lista")
        
        if isinstance(tipos, list):
            self.tipos = tipos
        else:
            raise ValueError("Tipos debe ser una lista")
    
    def esAplicable(self, producto, cantidad):
        return producto.codigo in self.codigos or producto.tipo in self.tipos

    def calcularDescuento(self, producto, cantidad):
        raise NotImplementedError("Este método debe ser implementado por las subclases")


class OfertaDescuento(Oferta):
    def __init__(self, descuento, codigos=[], tipos=[]):
        super().__init__(codigos=codigos, tipos=tipos)
        if not (0 <= descuento <= 100):
            raise ValueError("El descuento debe estar entre 0 y 100")
        self.descuento = descuento

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto, cantidad):
            return producto.precio * (self.descuento / 100) * cantidad
        return 0


class Oferta2x1(Oferta):
    def __init__(self, codigos=[], tipos=[]):
        super().__init__(codigos=codigos, tipos=tipos)

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto, cantidad):
            cantidad_gratis = cantidad // 2
            return producto.precio * cantidad_gratis
        return 0

In [4]:
## NO MODIFICAR ESTE CODIGO ##

p1 = Producto('1234', 'Coca Cola', 1000, 'gaseosa', 10)
p2 = Producto('1235', 'Oreo',      2300, 'galleta', 10)

o10d = OfertaDescuento(10, codigos=['1234'])
assert o10d.calcularDescuento(p1, 10) == 1000 
assert o10d.calcularDescuento(p1, 1) == 100

assert o10d.calcularDescuento(p2, 10) == 0

o2x1 = Oferta2x1(tipos=['galleta'])
assert o2x1.calcularDescuento(p1, 10) == 0

assert o2x1.calcularDescuento(p2, 1) == 0
assert o2x1.calcularDescuento(p2, 2) == 2300
assert o2x1.calcularDescuento(p2, 3) == 2300
assert o2x1.calcularDescuento(p2, 4) == 4600
assert o2x1.calcularDescuento(p2, 5) == 4600

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Catalogo
- `leer(archivo) `    : Carga los productos desde el archivo
- `guardar(archivo)`  : Guarda los productos en el archivo
- `agregar(producto)` : Agrega un producto al catalogo
- `buscar(codigo)`    : Busca un producto por codigo o None si no existe
- `registrarOferta(oferta)`  : Registra una oferta
- `buscarOferta(producto, cantidad)`: Busca una oferta por codigo o None si no existe
- `calcularDescuento(producto, cantidad)`: Calcula el descuento de una oferta
- `cantidadProductos`: Retorna la cantidad de productos en el catalogo
- `cantidadUnidades`: Retorna la cantidad de unidades en el catalogo
- `valorTotal`: retorna el valor total del catalogo sin descuentos
- `informe()`: retorna un string con el informe del catalogo 

In [5]:
import csv

class Catalogo:
    def __init__(self):
        self.productos = []
        self.ofertas = []

    @classmethod
    def leer(cls, archivo):
        catalogo = cls()
        with open(archivo, mode='r', newline='', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                producto = Producto(
                    codigo=row['codigo'],
                    nombre=row['nombre'],
                    precio=float(row['precio']),
                    tipo=row['tipo'],
                    cantidad=int(row['cantidad'])
                )
                catalogo.agregar(producto)
        return catalogo

    def guardar(self, archivo):
        with open(archivo, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=['codigo', 'nombre', 'precio', 'tipo', 'cantidad'])
            writer.writeheader()
            for producto in self.productos:
                writer.writerow({
                    'codigo': producto.codigo,
                    'nombre': producto.nombre,
                    'precio': producto.precio,
                    'tipo': producto.tipo,
                    'cantidad': producto.cantidad
                })

    def agregar(self, *productos):
        for producto in productos:
            self.productos.append(producto)

    def buscar(self, codigo):
        for producto in self.productos:
            if producto.codigo == codigo:
                return producto
        return None
    
    def reducirStock(self, codigo, cantidad):
        producto = self.buscar(codigo)
        if producto:
            if producto.cantidad >= cantidad:
                producto.cantidad -= cantidad
            else:
                raise ValueError(f"No hay suficiente stock de {producto.nombre}")
        else:
            raise ValueError(f"Producto con código {codigo} no encontrado en el catálogo.")

    def registrarOferta(self, oferta):
        self.ofertas.append(oferta)

    def buscarOferta(self, producto, cantidad):
        for oferta in self.ofertas:
            if oferta.esAplicable(producto, cantidad):
                return oferta
        return None

    def calcularDescuento(self, producto, cantidad):
        oferta = self.buscarOferta(producto, cantidad)
        if oferta:
            return oferta.calcularDescuento(producto, cantidad)
        return 0

    @property
    def cantidadProductos(self):
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        return sum(producto.cantidad for producto in self.productos)

    @property
    def valorTotal(self):
        return sum(producto.valorTotal() for producto in self.productos)
    
    def vender(self, producto, cantidad):
        if cantidad <= 0:
            raise ValueError("La cantidad vendida debe ser mayor que 0")

        producto_catalogo = self.buscar(producto.codigo)
        
        if producto_catalogo is None:
            raise ValueError(f"El producto con código {producto.codigo} no está en el catálogo")
        
        if producto_catalogo.cantidad < cantidad:
            raise ValueError(f"No hay suficientes unidades del producto {producto.nombre} para vender. Disponible: {producto_catalogo.cantidad}")

        producto_catalogo.cantidad -= cantidad

        print(f"Vendidas {cantidad} unidades de {producto.nombre}. Unidades restantes: {producto_catalogo.cantidad}")


    def informe(self):
        total_productos = self.cantidadProductos
        total_unidades = self.cantidadUnidades
        valor_total = self.valorTotal
        precio_promedio = valor_total / total_unidades if total_unidades > 0 else 0

        tipos = {}
        for producto in self.productos:
            if producto.tipo not in tipos:
                tipos[producto.tipo] = {'unidades': 0, 'valor_total': 0}
            tipos[producto.tipo]['unidades'] += producto.cantidad
            tipos[producto.tipo]['valor_total'] += producto.valorTotal()

        informe_tipos = ""
        for tipo, datos in tipos.items():
            unidades = datos['unidades']
            valor_total_tipo = datos['valor_total']
            precio_promedio_tipo = valor_total_tipo / unidades if unidades > 0 else 0
            informe_tipos += f"  - {tipo}              :  {unidades}u x $ {precio_promedio_tipo:.2f}\n"

        informe_ofertas = ""
        for oferta in self.ofertas:
            informe_ofertas += f"  - {oferta.descripcion}\n"

        return (f"INFORME CATALOGO\n"
                f"Cantidad de productos:   {total_productos}\n"
                f"Cantidad de unidades:    {total_unidades}\n"
                f"Precio Promedio:       $ {precio_promedio:.2f}\n"
                f"Valor total:           $ {valor_total:.2f}\n"
                f"Tipos de productos: "
                f"{informe_tipos}"
                f"Ofertas:\n"
                f"Oferta 2x1: "
                f"{informe_ofertas}")


In [6]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba del catálogo 

catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)

## Agregar productos al catalogo 
catalogo.agregar(p1)
catalogo.agregar(p2)
catalogo.agregar(p3)
catalogo.agregar(p4)

assert catalogo.cantidadProductos == 4
assert catalogo.cantidadUnidades == 100

assert catalogo.valorTotal == 167000

## Calcular descuentos segun las ofertas registradas
assert catalogo.calcularDescuento(p1, 5) == 0
assert catalogo.calcularDescuento(p2, 5) == 0

# Ofertas no acumulables 
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

assert catalogo.calcularDescuento(p1, 5) == 750
assert catalogo.calcularDescuento(p2, 5) == 0
assert catalogo.calcularDescuento(p3, 5) == 2400

assert catalogo.valorTotal == 167000.0
catalogo.guardar('catalogo-prueba.csv') ## Guardar datos antes de vender

# Vender afecta la cantidad de unidades y el valor total
catalogo.vender(p3, 3)   

# Verificar que el informe se genere correctamente

informe = catalogo.informe()
assert "Cantidad de productos: " in informe
assert "Cantidad de unidades: " in informe
assert "Precio Promedio: " in informe
assert "Valor total: " in informe
assert "Tipos de productos: " in informe
assert "gaseosa" in informe
assert "galleta" in informe
assert "Ofertas:" in informe 
assert "Oferta 2x1" in informe
assert catalogo.cantidadUnidades == 97
assert catalogo.valorTotal == 163400

# Buscar por código
assert catalogo.buscar('0001') == p1
assert catalogo.buscar('0002') == p2
assert catalogo.buscar('0099') is None 

# Recuperar los datos guardados  
c2 = Catalogo.leer('catalogo-prueba.csv')

assert c2.cantidadProductos == 4
assert c2.cantidadUnidades == 100

# Valor antes de guardar
assert c2.valorTotal == 167000.0

print("Prueba pasada exitosamente!")

Vendidas 3 unidades de Sonrisa. Unidades restantes: 27
Prueba pasada exitosamente!


# Cliente
- `nombre`: 1 a 100 caracteres
- `cuit`: 13 digitos (formato XX-XXXXXXXX-X)

In [7]:
class Cliente:
    def __init__(self, nombre, cuit):
        self.nombre = nombre
        self.cuit = cuit

    @property
    def nombre(self):
        return self._nombre

    @nombre.setter
    def nombre(self, value):
        if isinstance(value, str) and value != '':
            self._nombre = value
        else:
            return print("Ingrese un nombre por favor!")

    @property
    def cuit(self):
        return self._cuit

    @cuit.setter
    def cuit(self, value):
        cuit_sin_guiones = value.replace('-', '')
        if isinstance(cuit_sin_guiones, str) and cuit_sin_guiones.isdigit() and len(cuit_sin_guiones) == 11:
            self._cuit = value
        else:
            return print("Ingrese un CUIT válido!")


In [8]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Cliente #

c1 = Cliente('Juan Perez', '20-12345678-1')

assert c1.nombre == 'Juan Perez'
assert c1.cuit   == '20-12345678-1'

c1.nombre = ''
assert c1.nombre == 'Juan Perez' # Rechaza el valor vacio

c1.nombre = 'Juana Perez'        # Acepta el nuevo valor
assert c1.nombre == 'Juana Perez'

c1.cuit = '1234567890123'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

c1.cuit = 'CC-12345678-1'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

print("Prueba pasada exitosamente!")

Ingrese un nombre por favor!
Ingrese un CUIT válido!
Ingrese un CUIT válido!
Prueba pasada exitosamente!


In [9]:
from datetime import datetime

class Factura:
    _ultimo_numero = 100

    @classmethod
    def ultimaFactura(cls, numero):
        cls._ultimo_numero = numero

    @classmethod
    def nuevoNumero(cls):
        cls._ultimo_numero += 1
        return cls._ultimo_numero

    def __init__(self, catalogo, cliente):
        self.catalogo = catalogo
        self.numero = Factura.nuevoNumero()
        self.fecha = datetime.now().strftime("%Y-%m-%d")
        self.cliente = cliente
        self.productos = {}

    def agregar(self, producto, cantidad):
        if producto.codigo in self.productos:
            self.productos[producto.codigo]['cantidad'] += cantidad
        else:
            self.productos[producto.codigo] = {'producto': producto, 'cantidad': cantidad}
        
        try:
            catalogo.reducirStock(producto.codigo, cantidad)
        except ValueError as e:
            print(f'Error al reducir stock: {e}')
            return

    @property
    def cantidadProductos(self):
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        return sum(item['cantidad'] for item in self.productos.values())

    @property
    def subtotal(self):
        return sum(item['producto'].precio * item['cantidad'] for item in self.productos.values())

    @property
    def descuentos(self):
        total_descuentos = 0
        for item in self.productos.values():
            producto = item['producto']
            cantidad = item['cantidad']
            descuento = self.catalogo.calcularDescuento(producto, cantidad)  # Usa la referencia del catálogo
            total_descuentos += descuento
        return total_descuentos

    @property
    def total(self):
        return self.subtotal - self.descuentos

    def imprimir(self):
        detalles = f"Factura: {self.numero}\nFecha  : {self.fecha}\nCliente: {self.cliente.nombre} ({self.cliente.cuit})\n"
        for item in self.productos.values():
            producto = item['producto']
            cantidad = item['cantidad']
            subtotal = producto.precio * cantidad
            detalles += f"- {cantidad} {producto.nombre:<20} x ${producto.precio} = ${subtotal:.2f}\n"
            
            descuento = self.catalogo.calcularDescuento(producto, cantidad)
            if descuento > 0:
                detalles += f"      Descuento 10%: - ${descuento:.2f}\n"
                
            for oferta in catalogo.ofertas:
                if oferta.esAplicable(producto, cantidad):
                    detalles += f"      Oferta 2x1\n"

        detalles += f"                                             Subtotal:   ${self.subtotal:.2f}\n"
        detalles += f"                                             Descuentos: ${self.descuentos:.2f}\n"
        detalles += f"                                             -----------------------\n"
        detalles += f"                                             TOTAL:      ${self.total:.2f}\n"
        return detalles



In [10]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Factura #

# Creo un catálogo con productos
catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)
catalogo.agregar(p1,p2,p3,p4)

# Registro ofertas
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

# Creo un cliente
cliente = Cliente('Juan Perez', '20-12345678-9')

# Creo una factura
Factura.ultimaFactura(100)
assert Factura.nuevoNumero() == 101
assert Factura.nuevoNumero() == 102

f1 = Factura(catalogo, cliente)
f1.agregar(p1, 5)
f1.agregar(p3, 3)

assert f1.numero == 103
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades  == 8

# Agrega unidades de un producto ya agregado
f1.agregar(p1, 5)
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades == 13

assert f1.subtotal   == 18600
assert f1.descuentos == 2700.0
assert f1.total == 15900.0

impresion = f1.imprimir()

assert "Juan Perez" in impresion
assert "10 Coca Cola" in impresion
assert "Sonrisa" in impresion
assert "Descuento 10%" in impresion
assert "Oferta 2x1" in impresion
assert "TOTAL:" in impresion
assert "15900.00" in impresion

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [11]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de integración #

# Cargamos los datos
catalogo = Catalogo.leer('catalogo.csv')
juan  = Cliente('Juan Perez', '20-12345678-9')
maria = Cliente('Maria Lopez', '27-87654321-3')

o2x1 = Oferta2x1(tipos=['galleta'], codigos=['0002', '0003','0010'])
od20 = OfertaDescuento(20, codigos=['0001', '0002'], tipos=['gaseosa', 'arroz'])
od10 = OfertaDescuento(10, tipos=['fideo'])

catalogo.registrarOferta(o2x1)
catalogo.registrarOferta(od20)
catalogo.registrarOferta(od10)

# Controlo que la carga este correcta
assert catalogo.cantidadProductos == 30
assert catalogo.cantidadUnidades == 1000
assert catalogo.valorTotal == 2000000


Factura.ultimaFactura(10000)

# Crear una factura
f1 = Factura(catalogo, juan)
f1.agregar(catalogo.buscar('0001'), 5)
f1.agregar(catalogo.buscar('0002'), 3)
f1.agregar(catalogo.buscar('0003'), 2)

assert f1.numero == 10001
assert f1.cantidadProductos == 3
assert f1.cantidadUnidades == 10
assert f1.subtotal == 13450.0
assert f1.descuentos == 3890.0
assert f1.total == 9560.0

assert catalogo.cantidadUnidades == 990

# Crear otra factura
f2 = Factura(catalogo, maria)
f2.agregar(catalogo.buscar('0010'), 5)
f2.agregar(catalogo.buscar('0010'), 3)
f2.agregar(catalogo.buscar('0020'), 2)
f2.agregar(catalogo.buscar('0030'), 2)

assert f2.numero == 10002
assert f2.cantidadProductos == 3
assert f2.cantidadUnidades == 12
assert f2.subtotal == 23900.00
assert f2.descuentos == 8860.00
assert f2.total == 15040.00

assert catalogo.cantidadUnidades == 978

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!
